In [ ]:
import pandas as pd
import torch
from torch import nn, tensor, optim
from nltk.tokenize import word_tokenize
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from numpy import random
import numpy as np
import pickle
import urllib.request
import gzip
import json
import re
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
from collections import Counter


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/

Mounted at /content/drive
/content/drive/My Drive


In [ ]:
def parse(path):
    with open(path, 'r') as f:
        for line in f:
            yield json.loads(line)

# Initialize an empty list to store the sentences
para = []
data_path = 'data'

# Loop through the JSON data and extract the sentences
# for review in parse(data_path):
#     para.append(review['reviewText'])

num_reviews_to_read =  1000

for i, review in enumerate(parse(data_path)):
    para.append(review['reviewText'])
    
    # Stop reading the dataset once the desired number of reviews have been processed
    if i >= num_reviews_to_read - 1:
        break

        
pattern = re.compile(r'(?<![A-Z])[.!?]\s+')

# iterate over each paragraph and split it into sentences
sentences = []
for paragraph in para:
    sentences.extend(re.split(pattern, paragraph))
sentences = [re.sub(r'[^\w\s]+', '', sentence.strip().lower()) for sentence in sentences]

In [ ]:
word_freq = defaultdict(int)
for sent in sentences:
    sent = re.sub(r'[^\w\s\.\-]+', '', sent)
    sent = re.sub('([;:.,!?()])', r' \1 ', sent)
    words = sent.strip().split()  # split the sentence into a list of words
    for word in words:
        word_freq[word] += 1


In [ ]:
unk_count = sum(1 for count in word_freq.values() if count < 5)
word_freq['<UNK>'] = unk_count


In [ ]:
# Create vocabulary list and replace rare words with '<UNK>'
vocab = ['<UNK>']
rare_words = []
new_sentences = []
word_counts = {}


for sent in sentences:
    words = sent.strip().split()  # split the sentence into a list of words
    new_sent = []
    for word in words:
        # remove non-alphanumeric characters from each word
        word = re.sub(r'\W+', '', word)
        if word_freq[word.lower()] < 5:
            new_sent.append('<UNK>')
            rare_words.append(word.lower())
        else:
            new_sent.append(word)
    new_sentences.append(' '.join(new_sent))

# Add counts of rare words to <UNK>
for word in rare_words:
    if word in word_freq:
        if '<UNK>' not in word_counts:
            word_counts['<UNK>'] = 0
        word_counts['<UNK>'] += word_freq[word.lower()]
        del word_freq[word.lower()]


In [ ]:
print(new_sentences[55])

i <UNK> through a lot of it and wouldnt use it to show to a group as i had <UNK>


In [ ]:
unique_vocab = set()

for sentence in new_sentences:
    words = sentence.split()
    unique_vocab.update(words)

unique_vocab = list(unique_vocab)

vocabulary = sorted(list(vocab))
print(len(unique_vocab))

1457


In [ ]:
# Convert unique_vocab to a dictionary with indices as values
word_to_index = {word: i for i, word in enumerate(unique_vocab)}

# Create one hot vectors for each word in the vocabulary
one_hot_vectors = []
for word in unique_vocab:
    word_index = word_to_index[word]
    one_hot_vector = F.one_hot(torch.tensor(word_index), len(unique_vocab)).float()
    one_hot_vectors.append(one_hot_vector)

# Convert one_hot_vectors to a PyTorch tensor
one_hot_tensor = torch.stack(one_hot_vectors)
print(one_hot_tensor.shape)

torch.Size([1457, 1457])


In [ ]:
# Compute the probabilities of each word
word_probs = {word: count / len(unique_vocab) for word, count in word_freq.items()}

In [ ]:
# # Generate negative samples for each target word
# num_negative_samples = 5

# neg_samples = {}
# for target_word in unique_vocab:
#     neg_samples[target_word] = []
#     while len(neg_samples[target_word]) < num_negative_samples:
#         # Randomly sample a word
#         neg_word = random.choice(unique_vocab)
#         # Skip the target word and high-frequency words
#         if neg_word == target_word or word_probs[neg_word] > word_probs[target_word]:
#             continue
#         # Add the negative word to the list
#         neg_samples[target_word].append(neg_word)



In [ ]:
# Precompute the negative examples for each target word
num_negative_samples = 5

neg_samples = {}
for target_word in unique_vocab:
    # Get the frequency of the target word
    freq = word_freq[target_word]
    # Compute the unigram distribution raised to the 3/4 power
    weights = np.power(np.array([word_freq[word] for word in unique_vocab]), 0.75)
    weights /= np.sum(weights)
    # Sample negative examples according to the unigram distribution
    neg_words = np.random.choice(unique_vocab, size=num_negative_samples*freq, replace=True, p=weights)
    neg_words = [word for word in neg_words if word != target_word][:num_negative_samples]
    neg_samples[target_word] = neg_words


In [ ]:
# Create the input and output matrices
vocab_size = len(unique_vocab)
window_size = 2


# Create the input and output matrices
input_matrix = []
output_matrix = []
for sentence in new_sentences:
    words = sentence.split()
    for i, target_word in enumerate(words):
        # Ignore the target word if it is not in the vocabulary
        if target_word not in word_to_index:
            continue
        # Collect the context words
        context_words = []
        for j in range(max(0, i - window_size), min(len(words), i + window_size + 1)):
            if j != i and words[j] in word_to_index:
                context_words.append(words[j])

        # print(context_words)
        # Ignore the target word if there are no context words
        if not context_words:
            continue
        # Create one-hot encoding of the context words
        context_onehot = np.zeros(vocab_size)
        for context_word in context_words:
            # print(word_to_index[context_word])
            context_onehot[word_to_index[context_word]] = 1
            if context_word not in word_to_index:
                print("nay")
                continue
        # Add the input and output vectors to the matrices
        input_matrix.append(context_onehot)
        output_matrix.append(word_to_index[target_word])



In [ ]:
for i, (word, index) in enumerate(word_to_index.items()):
    if i < 25:
        print(f"{word}: {index}")
    else:
        break


jr: 0
turner: 1
exercise: 2
cut: 3
foldables: 4
pretty: 5
consider: 6
serious: 7
etc: 8
looking: 9
pilgrims: 10
end: 11
based: 12
its: 13
choir: 14
fan: 15
else: 16
went: 17
twist: 18
kind: 19
treasure: 20
face: 21
charming: 22
value: 23
sung: 24


In [ ]:
print(input_matrix[1])

[0. 0. 0. ... 0. 0. 0.]


In [ ]:
input_tensor = torch.tensor(input_matrix)
output_tensor = torch.tensor(output_matrix)
print(input_tensor.shape)
print(output_tensor.shape)

torch.Size([70696, 1457])
torch.Size([70696])


In [ ]:
print(output_matrix[55])

1309


In [ ]:
# Define one-hot encoding function
def one_hot(index, size):
    vec = [0] * size
    vec[index] = 1
    return vec


In [ ]:
class CBOWModel(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        super(CBOWModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.linear = nn.Linear(embedding_size, vocab_size)
    
    def forward(self, x):
        embedded = self.embedding(x)
        summed = torch.sum(embedded, dim=1)
        out = self.linear(summed)
        return out


In [ ]:
#hyperparameters
embedding_size = 100
learning_rate = 0.01
num_epochs = 10
#also includes window_size, vocab_size, num_negative_Samples

# Initialize the model, loss function, and optimizer
model = CBOWModel(vocab_size, embedding_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# Train the model
for epoch in range(num_epochs):
    running_loss = 0.0
    for i in range(len(input_matrix)):
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        inputs = torch.tensor(input_matrix[i]).unsqueeze(0)
        inputs = torch.tensor(input_matrix[i]).unsqueeze(0).long()
        target = torch.tensor(output_matrix[i]).unsqueeze(0)
        output = model(inputs)
        loss = criterion(output, target)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Update running loss
        running_loss += loss.item()
        
    # Print epoch loss
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(input_matrix)}")

Epoch 1, Loss: nan
Epoch 2, Loss: nan
Epoch 3, Loss: nan
Epoch 4, Loss: nan
Epoch 5, Loss: nan
Epoch 6, Loss: nan
Epoch 7, Loss: nan
Epoch 8, Loss: nan
Epoch 9, Loss: nan
Epoch 10, Loss: nan


In [ ]:
# # Get the embeddings from the model
# embeddings = model.embedding.weight.detach().numpy()
# # Convert embeddings to a dictionary
# embedding_dict = {f"embedding_{i}": embedding.tolist() for i, embedding in enumerate(embeddings)}

# # # Save the embeddings to a file
# # np.savetxt("cbow_emb.txt", embeddings)
# filename = 'cbow_emb.txt'

# # Open the file for writing
# with open(filename, 'w') as f:
#     # Write the contents of emb_list to the file
#     for key, value in embedding_dict.items():
#         f.write(f"{key}: {value}\n")



In [ ]:
# Get the embeddings from the model
embeddings = model.embedding.weight.detach().numpy()

# Convert embeddings to a dictionary with vocab as keys
embedding_dict = {unique_vocab[i]: embeddings[i].tolist() for i in range(len(unique_vocab))}

# Save the embeddings to a file
filename = 'cbow.txt'

# Open the file for writing
with open(filename, 'w') as f:
    # Write the contents of embedding_dict to the file
    for key, value in embedding_dict.items():
        f.write(f"{key}: {value}\n")
